In [1]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.lmstudio import LMStudio
import json
import dotenv
import phoenix as px
from llama_index.core import (
    Settings,
    set_global_handler,
)
import nest_asyncio
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import (
    get_qa_with_reference,
    get_retrieved_documents,
)
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from tqdm import tqdm
from llama_index.llms.bedrock import Bedrock
from llama_index.llms.bedrock_converse import BedrockConverse


dotenv.load_dotenv()
nest_asyncio.apply()

In [2]:
# pip install llama-index-llms-bedrock
# !pip install llama-index-llms-bedrock-converse

In [3]:
# session = px.launch_app()
# set_global_handler("arize_phoenix")

In [4]:
import boto3
import json
from datetime import datetime

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name=os.environ.get("AWS_REGION"),
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
)

In [3]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
import boto3
import json
from datetime import datetime

# Create a Bedrock Runtime client in the AWS Region of your choice.
# client = boto3.client("bedrock-runtime", region_name="us-east-1")

LITE_MODEL_ID = "amazon.nova-micro-v1:0"

# Define your system prompt(s).
system_list = [
            {
                "text": "Act as a creative writing assistant. When the user provides you with a topic, write a short story about that topic."
            }
]

# Define one or more messages using the "user" and "assistant" roles.
message_list = [{"role": "user", "content": [{"text": "A camping trip"}]}]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 500, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

request_body = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

start_time = datetime.now()

# Invoke the model with the response stream
response = client.invoke_model_with_response_stream(
    modelId=LITE_MODEL_ID, body=json.dumps(request_body)
)

request_id = response.get("ResponseMetadata").get("RequestId")
print(f"Request ID: {request_id}")
print("Awaiting first token...")

chunk_count = 0
time_to_first_token = None

# Process the response stream
stream = response.get("body")
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            # Print the response chunk
            chunk_json = json.loads(chunk.get("bytes").decode())
            # Pretty print JSON
            # print(json.dumps(chunk_json, indent=2, ensure_ascii=False))
            content_block_delta = chunk_json.get("contentBlockDelta")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = datetime.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")

                chunk_count += 1
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S:%f")
                # print(f"{current_time} - ", end="")
                print(content_block_delta.get("delta").get("text"), end="")
    print(f"Total chunks: {chunk_count}")
else:
    print("No response stream received.")

Request ID: 9d884c1a-f960-4faa-b113-21b8d9ecd57f
Awaiting first token...
Time to first token: 0:00:01.028882
Title: **Whispers of the Woods**

 the heart of autumn, when the leaves turned a kaleidoscope of reds and golds, Emily and her younger brother, Jake, embarked on a much-anticipated camping trip. Their parents had always told them stories of family adventures in the forest, but nothing could have prepared them for the magic they would find.

 Jake set up their tent, laughter echoing through the trees. filled with the crisp scent of pine needles and the distant murmur of a bubbling brook. As the sun dipped below the horizon, casting an amber glow over the clearing, Emily and

 where shadows danced under the canopy of stars. The brook's gentle flow guided them through the woods, its whispers mingling with the rustling leaves and the occasional hoot of an owl.

 walked, Emily spotted something glimmering among the leaves. She knelt down, brushing aside the foliage to reveal a small,

In [16]:
# pip install llama-index-embeddings-bedrock


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from llama_index.embeddings.bedrock import BedrockEmbedding

In [6]:
llm = BedrockConverse(
    model="amazon.nova-micro-v1:0",
    client=client,
) # Ollama(model="llama3.1:8b", request_timeout=60.0)
embedder = BedrockEmbedding(
    model_name="amazon.titan-embed-text-v2:0",
    client=client,
)

Settings.llm = llm
Settings.embed_model = embedder

In [7]:
llm.metadata.is_function_calling_model = True

In [8]:
llm.metadata

LLMMetadata(context_window=128000, num_output=512, is_chat_model=True, is_function_calling_model=True, model_name='amazon.nova-micro-v1:0', system_role=<MessageRole.SYSTEM: 'system'>)

In [9]:
llm.complete("hello")

CompletionResponse(text="Hello! How can I assist you today? Whether you have a question, need information, or just want to chat, I'm here to help.", additional_kwargs={'tool_calls': [], 'tool_call_id': [], 'status': []}, raw={'ResponseMetadata': {'RequestId': 'b5cbcda3-045a-455e-b94f-571ec34b5d06', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 13 Feb 2025 13:26:39 GMT', 'content-type': 'application/json', 'content-length': '301', 'connection': 'keep-alive', 'x-amzn-requestid': 'b5cbcda3-045a-455e-b94f-571ec34b5d06'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "Hello! How can I assist you today? Whether you have a question, need information, or just want to chat, I'm here to help."}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 1, 'outputTokens': 31, 'totalTokens': 32}, 'metrics': {'latencyMs': 321}}, logprobs=None, delta=None)

In [23]:
embedder.get_text_embedding("hello")

[-0.05515049770474434,
 0.04516850784420967,
 0.02720092423260212,
 0.006394712720066309,
 0.04142526164650917,
 -0.0359351672232151,
 0.009233341552317142,
 0.0314432717859745,
 0.017094159498810768,
 0.010793027468025684,
 -0.009794828481972218,
 0.03718291595578194,
 0.007611267734318972,
 -0.05415229871869087,
 -0.011104964651167393,
 -0.03543606773018837,
 -0.014972985722124577,
 -0.035685617476701736,
 0.04766400530934334,
 0.002916612895205617,
 -0.03393876925110817,
 0.016220735386013985,
 0.03393876925110817,
 -0.020587855949997902,
 -0.027700023725628853,
 -0.02607795037329197,
 -0.035186517983675,
 -0.03543606773018837,
 0.00948289129883051,
 -0.05714689567685127,
 0.04741445556282997,
 -0.0070497808046638966,
 -0.008235142566263676,
 -0.01397478673607111,
 0.0160959605127573,
 0.03793156519532204,
 0.019963981583714485,
 -0.02345767803490162,
 -0.012352713383734226,
 -0.01647028513252735,
 0.05440184846520424,
 -0.028199123218655586,
 -0.0061139692552387714,
 0.021086955443

In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [9]:
documents = SimpleDirectoryReader(input_files=["data/Phone_and_Internet_Allowance_Policy_abrd.pdf"]).load_data()

In [10]:
index = VectorStoreIndex.from_documents(documents, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=3)

In [11]:
response = query_engine.query("What does the term 'Social Media' refer to in the document?")
print(response)

In a professional setting, social media typically refers to online platforms where individuals or organizations share information, ideas, and experiences with a wide audience. These platforms often enable users to create and manage their own content, engage with others through comments and messages, and build relationships with people they may not know personally.


In [12]:
for source in response.source_nodes:
    print(source, end="\n")

Node ID: baa65c3d-7b9d-4e5b-bfb9-af0276bd2901
Text: EMPLOYEE  COMMUNICATIONS  POLICY    Document  owner:
Reference:    Author:   Communications  and  Human  Resources   Date:
14/08/2024   Approved  By:    Head  Human  Resources    Approval
Date:   14/01/2025   Classification:   Internal   Version:  1.0
Effective  Date:  14/01/2025    Record  of  Change   Name  Designation
Reason  for  Ch...
Score:  0.655

Node ID: 32e97649-cfac-427e-b904-a5b5e242638c
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggressive   or   inconsiderate,   it   is   recommended
that   ...
Score:  0.646

Node ID: ef8b46a3-3c17-4cde-953d-4a28ca43cbcc
Text:
Score:  0.631



In [13]:
response = query_engine.query("List some social media guidelines for usage?")
print(response)

for source in response.source_nodes:
    print(source, end="\n")

Here are some social media guidelines for usage:

1. Be ambassadors of the company by being respectful and professional online.
2. Employees are responsible for the content they put out or curate on their personal social media accounts, even if they are not affiliated with the company.
3. Avoid posting or sharing inflammatory, violent, or spreading misinformation.
4. Treat others with respect on social media, especially when responding to criticism of the company's work.
5. Refrain from responding to aggressive or inconsiderate comments.
6. Be mindful that personal social media accounts are public spaces and actions taken on them may have repercussions at work.
Node ID: 32e97649-cfac-427e-b904-a5b5e242638c
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggr

In [14]:
response = query_engine.query("What are professional websites?")
print(response)

for source in response.source_nodes:
    print(source, end="\n")

Websites that provide legitimate online services or resources for job or professional growth.
Node ID: 32e97649-cfac-427e-b904-a5b5e242638c
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggressive   or   inconsiderate,   it   is   recommended
that   ...
Score:  0.532

Node ID: bb36f5b9-4f31-4214-9ea2-9bee37aa12fe
Text: 4.2.8.  Helium  Health  understands  the  benefits  of  a
diverse  workforce  and  expects  that  every  employee  should
foster   an   understanding   of   others’   differences   in   order
to   create   an   environment   where   those   differences
contribute   to   a   better   organization.    5.  COMPANY  EMAIL
USE  5.1.  Definiti...
Score:  0.517

Node ID: d00f1d5b-d6ac-4428-8f96-2a01dddf33fc
Text: 5.5.  Email  Etiquette   5.5.1.  Al

## PDF Readers

In [15]:
# pip install PyMuPDF

In [16]:
documents[2]

Document(id_='14c9dc1d-296e-47e4-9b06-e108ed79ffde', embedding=None, metadata={'page_label': '3', 'file_name': 'Employee Communication Policy v1.pdf', 'file_path': 'data/Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 'file_size': 229997, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=" \nThis  Employee  Communication  Policy  (“ Policy ”)  contains  the  following  sections:   1.  SOCIAL  MEDIA  GUIDELINES   2.  PROFESSIONAL  WEBSITES  3.  THOUGHT  LEADERSHIP  PRACTICES  4.  INTERNAL  COMMUNICATION  PLATFORMS   5.  COMPANY  EMAIL  USE   1.  SOCIAL  

In [11]:
from llama_index.readers.file import PyMuPDFReader, PDFReader

In [70]:
# PyMuPDF Reader example
parser = PyMuPDFReader()
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_files=["./data/transport allowance.pdf"], file_extractor=file_extractor, encoding="utf8"
).load_data()

# PDF Reader with `SimpleDirectoryReader`
# parser = PDFReader()
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader(
#     "./data", file_extractor=file_extractor
# ).load_data()

In [75]:
documents[0].text

'One Global Medical Technology Ltd.\nRC: 1264820\nMEMO\nTO: Chief Operating Officer (COO), Finance Team, HR Team\nFROM: Head Human Resources\nDATE: 24th August 2023\nSUBJECT: Lagos Team Transport Allowance (Update on Travel Policy for Lagos Team Members)\nThis memo is being developed in response to the recent hike in transportation and petrol costs in Nigeria\nand is to apply to Helium Team Members who operate hybrid workdays of a minimum of 2 days in the\noffice. As at the time of this memo, this will apply to Lagos Team members only who are not receiving any\nkind of transport allowance. This memo is effective on the date stated, transport costs to the office within\nLagos will be subsidised by the Company at a monthly flat fee rate.\nPolicy Statement.\n●\nHelium Health will pay a Transport Allowance as a subsidised allowance for Lagos based employees\nonly, who are not receiving any kind of transport allowance.\n●\nAn allowance for transportation for employees and for personal use h

In [32]:
import unicodedata

In [76]:
normalized = unicodedata.normalize("NFKC", documents[0].text).strip().replace('\n', '').strip()
normalized

'One Global Medical Technology Ltd.RC: 1264820MEMOTO: Chief Operating Officer (COO), Finance Team, HR TeamFROM: Head Human ResourcesDATE: 24th August 2023SUBJECT: Lagos Team Transport Allowance (Update on Travel Policy for Lagos Team Members)This memo is being developed in response to the recent hike in transportation and petrol costs in Nigeriaand is to apply to Helium Team Members who operate hybrid workdays of a minimum of 2 days in theoffice. As at the time of this memo, this will apply to Lagos Team members only who are not receiving anykind of transport allowance. This memo is effective on the date stated, transport costs to the office withinLagos will be subsidised by the Company at a monthly flat fee rate.Policy Statement.●Helium Health will pay a Transport Allowance as a subsidised allowance for Lagos based employeesonly, who are not receiving any kind of transport allowance.●An allowance for transportation for employees and for personal use has been provided under the“Transpo

In [42]:
import re

In [46]:
re.split(r'[ ]{3,}', normalized)

['3.6. An employee whose device either becomes fully or partially inoperative or is otherwise unusable shall attempt to',
 'repair or replace their device or notify the HR Team within 5 days through any appropriate and available mode',
 'of contact.  4.PROCESSING 4.1. The department head will determine those employees in their department for whom the Phone and / or Internet',
 'Allowance is justified and the level of need for that employee. To request the allowance the department head',
 'will send the list of eligible employees to HR indicating the usage level.',
 '4.2. The employee will receive a Phone Allowance in the form of credit purchase to be paid monthly by the Finance',
 'department directly to the appropriate network providers; and Internet Allowance to be paid to the employee. 4.3. This allowance would come in before the end of the first week of every new month. 4.4. This allowance does not constitute an increase to base pay and will not be included in the calculation of',


In [97]:
from pydrive2.fs import GDriveFileSystem

# replace `root` with ID of a drive or directory and give service account access to it
fs = GDriveFileSystem("1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM", client_id="",
        client_secret="")

for fnames in fs.walk("1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/"):
    print(fnames, end="w")

('1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM', [], ['Memo_ Lagos Team Transport Allowance abridged.pdf', 'Phone_and_Internet_Allowance_Policy_abrd.pdf', 'Full_Policy_Jan_2021.pdf', 'Contract_Signatory_Authority_Policy.pdf', 'Leave_Handover_Form_shrd.docx', 'Intellectual_Property_and_Copyright_Compliance_Policy_17062022.pdf', 'Guideline_document_on_the_use_of_Generative_AI_clean_28072023.pdf', 'Helium_Health_Employee_Handbook_V1.0_20210825 (2).pdf', 'Full_Policy_updated_Aug_2023 (2).pdf', 'Memo__Meeting_and_Email_Guidelines_2023.pdf', 'HH_ISMS0502_POL_Information_Security_Management_System_Policy_V3.0_20240710_FIN (1).docx', 'HH_ISMSA.5.1_POL_Policies_for_Information_Security_V3.0_20240820_FIN (1).docx', 'Org Structure shared Nov 2024.pdf', 'Employee Communication Policy v1.pdf'])w

In [112]:
from llama_index.readers.file import DocxReader

In [117]:
reader = SimpleDirectoryReader(
    input_files=['1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/Leave_Handover_Form_shrd.docx'],
    fs=fs,
    file_extractor={".pdf": DocxReader()}
    # recursive=True,  # recursively searches all subdirectories
)

In [118]:
docs = reader.load_data(fs=fs)

Failed to load file 1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/Leave_Handover_Form_shrd.docx with error: File is not a zip file. Skipping...


In [111]:
docs[0]

Document(id_='ada6e74d-2c4a-4e20-889b-4453198de206', embedding=None, metadata={'page_label': '1', 'file_name': 'Memo_ Lagos Team Transport Allowance abridged.pdf', 'file_path': '1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/Memo_ Lagos Team Transport Allowance abridged.pdf', 'file_type': 'application/pdf', 'file_size': 109667}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='OneGlobal Medical TechnologyLtd.RC:1264820\nMEMO\nTO: Chief OperatingOfficer(COO), FinanceTeam, HRTeamFROM: HeadHumanResourcesDATE: 24thAugust2023SUBJECT: LagosTeamTransportAllowance(UpdateonTravel PolicyforLagosTeamMembers)\nThis memois being developedinresponse tothe recent hike

### Clean Text 

In [47]:
def clean_text(text):
    # normalize text for unicode
    clean_text = unicodedata.normalize("NFKD", text).strip()
    clean_text = clean_text.replace('\n', '')
    clean_text = " ".join(re.split(r'[ ]{3,}', normalized))
    return clean_text


In [48]:
clean_text(documents[2].text)

'3.6. An employee whose device either becomes fully or partially inoperative or is otherwise unusable shall attempt to repair or replace their device or notify the HR Team within 5 days through any appropriate and available mode of contact.  4.PROCESSING 4.1. The department head will determine those employees in their department for whom the Phone and / or Internet Allowance is justified and the level of need for that employee. To request the allowance the department head will send the list of eligible employees to HR indicating the usage level. 4.2. The employee will receive a Phone Allowance in the form of credit purchase to be paid monthly by the Finance department directly to the appropriate network providers; and Internet Allowance to be paid to the employee. 4.3. This allowance would come in before the end of the first week of every new month. 4.4. This allowance does not constitute an increase to base pay and will not be included in the calculation of percentage increases to bas

## Document Splitters

In [20]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=10,
)

print(f"Len documents: ", len(documents))
nodes = splitter.get_nodes_from_documents(documents)
print(f"Len nodes: ", len(nodes))

Len documents:  7
Len nodes:  18


In [21]:
print(nodes[4])

Node ID: 858eb49e-695b-42d6-b42b-8669725369d8
Text: 1.2.4​ Employees' personal social media accounts are online safe
zones and are not policed by Helium Health. It is  however the duty of
the employee to remember that these are very public spaces and actions
taken on these  spaces may have repercussions at Helium Health if they
are considered to be egregious.  1.2.5​ Members of employees are to
a...


In [22]:
index = VectorStoreIndex(nodes=nodes, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

In [23]:
response = query_engine.query("What does the term 'Social Media' refer to in the document?")
print(response, end="\n")

for source in response.source_nodes:
    print(source, end="\n")

The term "Social Media" refers to online platforms which aid interactions among people, including but not limited to X (formerly Twitter), Facebook, Tiktok, Snapchat, or any other platform by which individuals engage for the purpose(s) of creating or curating content.
Node ID: 9193f6d4-133b-431f-afdb-c513aefb9ffa
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hitherto...
Score:  0.748

Node ID: b42b40ac-b7a7-41d5-b38b-5c3facd82e14
Text: If  you provide us with any contact information for customer
service purposes, it will only be used to  manage your enquiry and
will not be used for any other reason.”  OR  “Any personal data shared
in public by the user on social platforms is shared at their own

In [24]:
response = query_engine.query("List some social media guidelines for usage?")
print(response, end="\n\n")

for source in response.source_nodes:
    print(source, end="\n")

The company encourages members of the team to be ambassadors of the company on social media platforms. When engaging online, employees are viewed by customers, partners, and other external parties, placing an even greater responsibility on them.

Employees are responsible for the content they put out or curate on these platforms, and are prohibited from posting or sharing anything that may be considered inflammatory, inciting of violence (in any form) or spreading of misinformation.

Node ID: 9193f6d4-133b-431f-afdb-c513aefb9ffa
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hitherto...
Score:  0.768

Node ID: b42b40ac-b7a7-41d5-b38b-5c3facd82e14
Text: If  you provide us with any contact informa

In [25]:
response = query_engine.query("What are professional websites?")
print(response, end="\n\n")

for source in response.source_nodes:
    print(source, end="\n")

Professional networking platforms such as LinkedIn, Angel List, Crunchbase, and others that allow individuals to share or curate content related to their work or professional progress.

Node ID: b42b40ac-b7a7-41d5-b38b-5c3facd82e14
Text: If  you provide us with any contact information for customer
service purposes, it will only be used to  manage your enquiry and
will not be used for any other reason.”  OR  “Any personal data shared
in public by the user on social platforms is shared at their own
risk.”    1.3.2​ The Admin(s) should delete (where possible) any
personal data that...
Score:  0.662

Node ID: 9193f6d4-133b-431f-afdb-c513aefb9ffa
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hithert

In [26]:
from llama_index.core.node_parser import SemanticSplitterNodeParser

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embedder
)

print(f"Len documents: ", len(documents))
nodes = splitter.get_nodes_from_documents(documents)
print(f"Len nodes: ", len(nodes))

Len documents:  7
Len nodes:  12


In [27]:
documents[2]

Document(id_='3f7f2aea-22c3-425f-a9fb-7cf68d4af628', embedding=None, metadata={'file_path': '/Users/sasuusen/Projects/Helium/lola/notebooks/data/Employee Communication Policy v1.pdf', 'file_name': 'Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 'file_size': 229997, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-21', 'total_pages': 7, 'source': '3'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=" \nThis Employee Communication Policy (“Policy”) contains the following sections: \n \n1.\u200b\nSOCIAL MEDIA GUIDELINES  \n2.\u200b\nPROFESSIONAL WEBSITES \n3.\u200b\nTHOUGHT LEADERSHIP PRACTICES \n4.\u200b\

In [28]:
nodes[3]

TextNode(id_='b7430188-5e5e-4de6-85bc-2b68c4aece98', embedding=None, metadata={'file_path': '/Users/sasuusen/Projects/Helium/lola/notebooks/data/Employee Communication Policy v1.pdf', 'file_name': 'Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 'file_size': 229997, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-21', 'total_pages': 7, 'source': '3'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3f7f2aea-22c3-425f-a9fb-7cf68d4af628', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/sasuusen/Projects/Helium/lola/notebooks/data/Employee Communication Policy v1.pdf', 'file_name': 'Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 

In [29]:
index = VectorStoreIndex(nodes=nodes, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

In [30]:
response = query_engine.query("What can i post on my personal twitter?")
print(response, end="\n")

for source in response.source_nodes:
    print(source, end="\n")

You should refrain from posting anything that may be considered inflammatory, inciting of violence, spreading of misinformation, or lacking in basic human decency.
Node ID: 13c4b871-6e93-4c9c-a478-5fab39d12db4
Text: 1.2.8​ Employees are not to share, reshare or endorse any news
about Helium Health on their social media without  confirming its
authenticity from line managers or the members of the Communications
team as they may be  false or unconfirmed.    1.3​Data Protection
1.3.1​ Where a social media user expresses their intention to share
their personal...
Score:  0.611

Node ID: ba19a985-ce17-48a0-a19a-57c882b5c000
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hitherto...
Score:  0.608

Nod

In [33]:
response.response

'You should refrain from posting anything that may be considered inflammatory, inciting of violence, spreading of misinformation, or lacking in basic human decency.'

## 

In [119]:
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.index_store.redis import RedisIndexStore
from redisvl.schema import IndexSchema
from llama_index.core.storage import StorageContext

In [120]:
REDIS_HOST = "localhost"
REDIS_PORT = 6379
LLM = Ollama(model="llama3-groq-tool-use", request_timeout=60.0)
EMBED_MODEL = OllamaEmbedding(
    model_name="nomic-embed-text",
)
DOC_STORE = RedisDocumentStore.from_host_and_port(
    REDIS_HOST, REDIS_PORT, namespace="lola_document_store"
)
VECTOR_STORE = RedisVectorStore(
    schema=IndexSchema.from_dict(dict(json.load(open("../custom_redis_vector_schema.json", "r")))),
    redis_url=f"redis://{REDIS_HOST}:{REDIS_PORT}",
)
INDEX_STORE = RedisIndexStore.from_host_and_port(
    host=REDIS_HOST, port=REDIS_PORT, namespace="lola_index"
)
STORAGE_CONTEXT = StorageContext.from_defaults(
    index_store=INDEX_STORE,
    docstore=DOC_STORE,
)

14:46:08 redisvl.index.index INFO   Index already exists, not overwriting.


In [150]:
from llama_index.core import load_index_from_storage, load_indices_from_storage

In [151]:
vector_index = load_indices_from_storage(
    STORAGE_CONTEXT
)

In [160]:
vector_index[2].summary

'None'

In [161]:
vector_engine = vector_index[1].as_retriever(similarity_top_k=3)

In [162]:
await vector_engine.aretrieve("phone and internet policy?")

[]

## Evaluation

In [18]:
from llama_index.llms.lmstudio import LMStudio

In [19]:
from phoenix.evals import LiteLLMModel
from litellm import completion

/Users/sasuusen/miniconda3/envs/Helium/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [20]:
import os

os.environ['LM_STUDIO_API_BASE'] = "http://127.0.0.1:1234/v1"
os.environ['LM_STUDIO_API_KEY'] = "sk-123"

In [21]:
eval_model = LiteLLMModel(model="lm_studio/phi4-latest", _verbose=True)

In [22]:
response = eval_model("Hey there, what is 2+2?")
print(str(response))

2 + 2 equals 4. If you have any more questions or need further assistance, feel free to ask!


In [23]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [24]:
queries_df

,input,output,reference
context.span_id,,,
78c28db07ff9886c,What does the term 'Social Media' refer to in ...,"In the document, ""social media"" refers to onli...",EMPLOYEE COMMUNICATIONS POLICY Document ...
2afccf2582b4cae4,List some social media guidelines for usage?,The organization encourages team members to ac...,1.2.5 Members of employees are to always...
b886d4efd22d34fb,What are professional websites?,Professional websites refer to online platform...,1.2.5 Members of employees are to always...
35fab9bee5eb6300,What does the term 'Social Media' refer to in ...,"The term ""Social Media"" in the document refers...",Definitions 1.1.1. The usage of the term...
3354bc4371077c9f,List some social media guidelines for usage?,Employees are encouraged to act as ambassadors...,Definitions 1.1.1. The usage of the term...
1e5c387c5d636d65,What are professional websites?,Professional websites refer to platforms desig...,Definitions 2.1.1. The usage of the term...


In [25]:
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
530af4522d6d8273 0                  d51ccb08c0fedf13cb7faadc0ef8709b   
                 1                  d51ccb08c0fedf13cb7faadc0ef8709b   
                 2                  d51ccb08c0fedf13cb7faadc0ef8709b   
fbefcb4215e8cdc8 0                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
                 1                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
                 2                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
ab83ed62670696ff 0                  9d6a962b94adafa8b989b7b5939b443c   
                 1                  9d6a962b94adafa8b989b7b5939b443c   
                 2                  9d6a962b94adafa8b989b7b5939b443c   
fed4ecdba15db354 0                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 1                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 2                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 3                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 4                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
aa15c5e3b65359af 0                  de3849188cb84f02bf780b556c9b02eb   
                 1                  de3849188cb84f02bf780b556c9b02eb   
                 2                  de3849188cb84f02bf780b556c9b02eb   
                 3                  de3849188cb84f02bf780b556c9b02eb   
                 4                  de3849188cb84f02bf780b556c9b02eb   
a9e42c0917f8a29f 0                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 1                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 2                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 3                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 4                  1ef2fc386563f4bfb7f2cccc42a93edd   

                                                                                input  \
context.span_id  document_position                                                      
530af4522d6d8273 0                  What does the term 'Social Media' refer to in ...   
                 1                  What does the term 'Social Media' refer to in ...   
                 2                  What does the term 'Social Media' refer to in ...   
fbefcb4215e8cdc8 0                       List some social media guidelines for usage?   
                 1                       List some social media guidelines for usage?   
                 2                       List some social media guidelines for usage?   
ab83ed62670696ff 0                                    What are professional websites?   
                 1                                    What are professional websites?   
                 2                                    What are professional websites?   
fed4ecdba15db354 0                  What does the term 'Social Media' refer to in ...   
                 1                  What does the term 'Social Media' refer to in ...   
                 2                  What does the term 'Social Media' refer to in ...   
                 3                  What does the term 'Social Media' refer to in ...   
                 4                  What does the term 'Social Media' refer to in ...   
aa15c5e3b65359af 0                       List some social media guidelines for usage?   
                 1                       List some social media guidelines for usage?   
                 2                       List some social media guidelines for usage?   
                 3                       List some social media guidelines for usage?   
                 4                       List some social media guidelines for usage?   
a9e42c0917f8a29f 0                                    What are professional websites?   
                 1                                    What are professional websites?   
                 2                                    What are professional websites?   
                 3                         

In [26]:
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

In [27]:
%%time
hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
    # concurrency=1
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
    # concurrency=1
)[0]

I0000 00:00:1737543835.687621    9601 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


run_evals |          | 0/12 (0.0%) | ⏳ 00:00<? | ?it/s


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
Requeuing...
Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
Requeuing...
Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Er

run_evals |          | 0/24 (0.0%) | ⏳ 00:00<? | ?it/s


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: raised litellm.BadRequestError: Lm_studioException - Error code: 400 - {'error': 'The model has crashed without additional information. (Exit code: 11)'}
Requeuing...
CPU times: user 20.5 s, sys: 4.17 s, total: 24.7 s
Wall time: 51min 15s


In [28]:
px.Client().log_evaluations(
    SpanEvaluations(
        eval_name="Hallucination", dataframe=hallucination_eval_df
    ),
    SpanEvaluations(
        eval_name="QA Correctness", dataframe=qa_correctness_eval_df
    ),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)